# Fazendo as previsões com o modelo escolhido

Aplicaremos o mesmo tratamento aos dados realizado na modelagem no dataset que será submetido à predição

In [1]:
#Importando as bibilotecas
import pandas as pd
import numpy as np
# configurando pandas para mostrar todas as linhas e colunas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) 
# configurando pandas para não mostrar notação científica para números
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
# Carregando o dataset de teste
dataset_teste = pd.read_csv('../data/cars_test.csv', encoding='utf-16',  sep ='\t', index_col=0)

In [3]:
dataset_teste.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9862 entries, 13518783164498355150900635905895481162 to 332784980027104367393384305973418103109
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   num_fotos                9802 non-null   float64
 1   marca                    9862 non-null   object 
 2   modelo                   9862 non-null   object 
 3   versao                   9862 non-null   object 
 4   ano_de_fabricacao        9862 non-null   int64  
 5   ano_modelo               9862 non-null   float64
 6   hodometro                9862 non-null   float64
 7   cambio                   9862 non-null   object 
 8   num_portas               9862 non-null   int64  
 9   tipo                     9862 non-null   object 
 10  blindado                 9862 non-null   object 
 11  cor                      9862 non-null   object 
 12  tipo_vendedor            9862 non-null   object 
 13  cidade_vend

In [4]:
# Retirar o nome do Estado e deixar só a sigla
dataset_teste['UF'] = dataset_teste['estado_vendedor'].apply(lambda x: x[-3:-1])

In [5]:
# Deixando a base original para aplicação dos preços após a predição
dataset_original = dataset_teste[:]

In [6]:
dataset_teste.veiculo_único_dono.fillna('', inplace=True)

In [7]:
# Alteração para os dados NaN
dataset_teste.revisoes_concessionaria.fillna(0, inplace=True)
dataset_teste.garantia_de_fábrica.fillna(0, inplace=True)
dataset_teste.revisoes_dentro_agenda.fillna(0, inplace=True)
dataset_teste.veiculo_alienado.fillna(0, inplace=True)
dataset_teste.veiculo_único_dono.fillna(0, inplace=True)

In [8]:
#Importando o LabelEnconder para transformar as variáveis categóricas em numéricas
from sklearn.preprocessing import LabelEncoder

In [9]:
leBlindado = LabelEncoder()
dataset_teste.blindado = leBlindado.fit_transform(dataset_teste.blindado)

In [10]:
leCambio = LabelEncoder()
dataset_teste.cambio = leCambio.fit_transform(dataset_teste.cambio)

In [11]:
leUF = LabelEncoder()
dataset_teste.UF = leUF.fit_transform(dataset_teste.UF)

In [12]:
leVersao = LabelEncoder()
dataset_teste.versao = leVersao.fit_transform(dataset_teste.versao)

In [13]:
leModelo = LabelEncoder()
dataset_teste.modelo = leVersao.fit_transform(dataset_teste.modelo)

In [14]:
leTipo = LabelEncoder()
dataset_teste.tipo = leTipo.fit_transform(dataset_teste.tipo)

In [15]:
dataset_teste.ano_modelo = dataset_teste.ano_modelo.astype(int)

In [16]:
# Encoding das variáveis e tranformação dos tipos
dataset_teste.revisoes_concessionaria.replace('Todas as revisões feitas pela concessionária', 1, inplace=True)
dataset_teste.revisoes_concessionaria = dataset_teste.revisoes_concessionaria.astype(int)

In [17]:
dataset_teste.garantia_de_fábrica.replace('Garantia de fábrica', 1, inplace=True)
dataset_teste.garantia_de_fábrica.replace = dataset_teste.revisoes_concessionaria.astype(int)

In [18]:
dataset_teste.veiculo_único_dono.replace('Único dono', 1, inplace=True)

In [19]:
dataset_teste.veiculo_único_dono.replace('', 0, inplace=True)
dataset_teste.veiculo_único_dono = dataset_teste.veiculo_único_dono.astype(int)

In [20]:
# Definindo o stream de dados a ser enviado ao modelo para predição
X = dataset_teste[['modelo', 'versao', 'ano_de_fabricacao', 'ano_modelo', 'cambio', 'tipo', 'blindado', 'veiculo_único_dono', 'garantia_de_fábrica', 'UF']]

In [21]:
# Normalização - MinMaxScaler, Padronização - StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Criar scaler
scaler = MinMaxScaler()

In [22]:
colunas = X.columns

# Definindo a escala
X[colunas] = scaler.fit_transform(X)
#X[colunas] = scaler.transform(X)

X.shape

E:\msalv\Anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


(9862, 10)

In [23]:
# Carregando o modelo campeão
import pickle
with open('modeloPreditor.pkl', 'rb') as modelo:
    model = pickle.load(modelo)

In [24]:
# Realizando a predição de preços com o modelo
precos_preditos = model.predict(X)

In [25]:
dataset_original['preco'] = precos_preditos

E:\msalv\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
dataset_original.tipo.unique()

array(['Hatchback', 'Picape', 'Utilitário esportivo', 'Sedã', 'Cupê',
       'Perua/SW', 'Minivan', 'Conversível'], dtype=object)

In [27]:
dataset_original.head()

,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,tipo_vendedor,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,UF,preco
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13518783164498355150900635905895481162,8.00,NISSAN,VERSA,1.6 16V FLEXSTART V-DRIVE MANUAL,2021,2021.00,20258.00,Manual,4,Hatchback,N,Branco,PF,Rio de Janeiro,São Paulo (SP),Pessoa Física,False,False,False,Aceita troca,,0,IPVA pago,Licenciado,Garantia de fábrica,0,0.00,SP,102531.23
299896161723793383246777788797566040330,18.00,FIAT,STRADA,1.4 MPI WORKING CS 8V FLEX 2P MANUAL,2021,2021.00,53045.00,Manual,2,Picape,N,Branco,PJ,Palmas,Amazonas (AM),Loja,False,False,False,Aceita troca,Único dono,0,IPVA pago,Licenciado,0,0,0.00,AM,104984.31
316180649972302128246133616457018378621,8.00,AUDI,Q5,2.0 TFSI GASOLINA BLACK S TRONIC,2018,2019.00,32486.00,Automática,4,Utilitário esportivo,N,Branco,PF,Goiânia,Goiás (GO),Pessoa Física,False,False,False,Aceita troca,,0,IPVA pago,NaN,0,0,0.00,GO,261206.50
222527157104148385909188217274642813298,16.00,CHEVROLET,CRUZE,1.4 TURBO LT 16V FLEX 4P AUTOMÁTICO,2019,2020.00,24937.00,Automática,4,Sedã,N,Cinza,PJ,Presidente Prudente,São Paulo (SP),Loja,False,False,False,Aceita troca,Único dono,0,IPVA pago,Licenciado,0,0,0.00,SP,106681.54
160460343059850745858546502614838368036,8.00,FORD,ECOSPORT,1.5 TI-VCT FLEX SE AUTOMÁTICO,2019,2019.00,62074.00,Automática,4,Sedã,N,Branco,PJ,Limeira,São Paulo (SP),Loja,False,False,False,Aceita troca,,0,IPVA pago,Licenciado,0,0,0.00,SP,112364.78


In [28]:
# Visualização estatística inicial do dataset
dataset_original.describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,9802.00,9862.00,9862.00,9862.00,9862.00,9862.00,9862.00
mean,10.32,2016.72,2017.80,58237.21,3.94,0.00,134465.59
std,3.46,4.15,2.68,32487.02,0.33,0.00,68206.77
min,8.00,1988.00,2007.00,100.00,2.00,0.00,-2126.96
25%,8.00,2015.00,2016.00,31323.25,4.00,0.00,87505.79
50%,8.00,2018.00,2018.00,56742.00,4.00,0.00,115516.94
75%,14.00,2019.00,2020.00,81784.00,4.00,0.00,168699.57
max,21.00,2022.00,2023.00,381728.00,4.00,0.00,835440.25


## Respondendo às perguntas de negócio

### Qual o melhor estado na base para se vender um carro de marca popular? Por que?
Ao avaliar o preço médio de venda dos veículos considerados populares, entre os estados de maior número de ofertas com preços máximos, verificamos que o Estado do Paraná possui os mais altos preços de venda para esses modelos. Não utilizamos a feature 'marca' em função da distribuição conter veículos de luxo para a mesma marca.

In [29]:
# Avaliação do dataset por marca
dataset_original.marca.unique()

array(['NISSAN', 'FIAT', 'AUDI', 'CHEVROLET', 'FORD', 'JEEP', 'TOYOTA',
       'MITSUBISHI', 'MERCEDES-BENZ', 'PEUGEOT', 'HYUNDAI', 'VOLKSWAGEN',
       'LAND ROVER', 'PORSCHE', 'HONDA', 'RENAULT', 'BMW', 'KIA', 'VOLVO',
       'JAGUAR', 'SSANGYONG', 'LEXUS', 'CHERY', 'RAM', 'MINI', 'CITROËN',
       'JAC', 'SUZUKI', 'TROLLER', 'SMART', 'MASERATI', 'CHRYSLER',
       'FERRARI', 'LIFAN', 'SUBARU', 'EFFA', 'ALFA ROMEO', 'LAMBORGHINI',
       'DODGE'], dtype=object)

In [30]:
# Selecionamos as marcas mais comuns no mercado brasileiro
marcas_populares = ['JAC', 'FIAT', 'CHEVROLET', 'FORD', 'PAMPA', 'VOLKSWAGEN', 'RENAULT', 'LIFAN']
carros_filtrados = dataset_original[dataset_original['marca'].isin(marcas_populares)]

In [31]:
# Optamos por utilizar a feature 'modelo' em função da seleção mostrar veículos de luxo para marcas consideradas populares.
carros_filtrados.head(10)

,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,tipo_vendedor,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,UF,preco
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
299896161723793383246777788797566040330,18.00,FIAT,STRADA,1.4 MPI WORKING CS 8V FLEX 2P MANUAL,2021,2021.00,53045.00,Manual,2,Picape,N,Branco,PJ,Palmas,Amazonas (AM),Loja,False,False,False,Aceita troca,Único dono,0,IPVA pago,Licenciado,0,0,0.00,AM,104984.31
222527157104148385909188217274642813298,16.00,CHEVROLET,CRUZE,1.4 TURBO LT 16V FLEX 4P AUTOMÁTICO,2019,2020.00,24937.00,Automática,4,Sedã,N,Cinza,PJ,Presidente Prudente,São Paulo (SP),Loja,False,False,False,Aceita troca,Único dono,0,IPVA pago,Licenciado,0,0,0.00,SP,106681.54
160460343059850745858546502614838368036,8.00,FORD,ECOSPORT,1.5 TI-VCT FLEX SE AUTOMÁTICO,2019,2019.00,62074.00,Automática,4,Sedã,N,Branco,PJ,Limeira,São Paulo (SP),Loja,False,False,False,Aceita troca,,0,IPVA pago,Licenciado,0,0,0.00,SP,112364.78
262727202368986244371977784343233436999,17.00,CHEVROLET,CRUZE,1.8 LT 16V FLEX 4P AUTOMÁTICO,2013,2014.00,88982.00,Automática,4,Sedã,N,Branco,PJ,São José do Rio Preto,São Paulo (SP),Loja,True,True,False,Aceita troca,,0,NaN,NaN,0,0,0.00,SP,65473.27
185631381169816366226494379020452029389,8.00,FORD,BRONCO SPORT,2.0 ECOBOOST GASOLINA WILDTRAK 4X4 SELECTSHIFT,2021,2021.00,12621.00,Automática,4,Utilitário esportivo,N,Cinza,PF,São José do Rio Preto,São Paulo (SP),Pessoa Física,False,False,False,Aceita troca,Único dono,0,IPVA pago,Licenciado,Garantia de fábrica,0,0.00,SP,257083.00
55430829679256880238551444522175018156,8.00,FIAT,TORO,1.8 16V EVO FLEX FREEDOM AT6,2019,2019.00,72641.00,Automática,4,Picape,N,Branco,PF,Gravataí,Rio Grande do Sul (RS),Pessoa Física,False,False,False,Aceita troca,,0,NaN,NaN,0,0,0.00,RS,129349.05
215317645068876070368783807654911699750,8.00,CHEVROLET,TRAILBLAZER,2.8 PREMIER 4X4 16V TURBO DIESEL 4P AUTOMÁTICO,2019,2020.00,32212.00,Automática,4,Utilitário esportivo,N,Cinza,PF,Belo Horizonte,Paraná (PR),Pessoa Física,False,False,False,Aceita troca,Único dono,Todas as revisões feitas pela concessionária,IPVA pago,NaN,Garantia de fábrica,Todas as revisões feitas pela agenda do carro,0.00,PR,265491.59
81597227204971961501733390453078413740,8.00,FORD,RANGER,3.2 LIMITED 4X4 CD 20V DIESEL 4P AUTOMÁTICO,2017,2017.00,95609.00,Automática,4,Picape,N,Branco,PF,Campinas,São Paulo (SP),Pessoa Física,False,False,False,Aceita troca,,Todas as revisões feitas pela concessionária,IPVA pago,Licenciado,0,0,0.00,SP,260875.61
185526399598537813925895625787601650168,13.00,FIAT,TORO,1.8 16V EVO FLEX FREEDOM OPEN EDITION AT6,2016,2017.00,92456.00,Automática,4,Picape,N,Branco,PJ,Rio de Janeiro,São Paulo (SP),Loja,True,False,False,Aceita troca,,0,IPVA pago,NaN,0,0,0.00,SP,109001.16


In [32]:
# Optamos por realizar a seleção por modelo
dataset_original.modelo.unique() 

array(['VERSA', 'STRADA', 'Q5', 'CRUZE', 'ECOSPORT', 'Q3', 'COMPASS',
       'HILUX', 'BRONCO SPORT', 'A3', 'PAJERO FULL', 'TORO',
       'TRAILBLAZER', 'RANGER', 'C 63 AMG', '2008', 'A5', 'CRETA',
       'MUSTANG', 'FOX', 'UNO', 'AMAROK', 'PRISMA', '3008',
       'DISCOVERY SPORT', 'POLO', 'OUTLANDER', 'ASX', 'PALIO', 'GOLF',
       '718', 'CRV', 'T-CROSS', 'C 250', 'DUSTER', 'DUSTER OROCH',
       'GLK 220', 'RS6', 'GOL', 'HB20', 'TRACKER', 'MONTANA', '320i',
       'S10', '220i', 'CIVIC', 'SENTRA', 'KWID', 'HR-V', 'ARGO',
       'SPACEFOX', 'CAYENNE', 'FIT', 'C 180', 'X3', 'SPORTAGE', 'A 200',
       'XC90', 'E 300', 'AZERA', 'FUSCA', 'JETTA', 'FOCUS', 'SL 63 AMG',
       'KA +', 'ONIX', 'X1', '325i', 'CAPTUR', 'X5', 'CRONOS', 'IX35',
       'MACAN', 'KICKS', 'HILUX SW4', 'F-PACE', 'KORANDO', 'RENEGADE',
       'LOGAN', 'NX 200T', 'COROLLA', 'B 200', 'NIVUS', 'VIRTUS',
       'CLA 45 AMG', 'EQUUS', 'SANTA FÉ', 'ELANTRA', 'TIGUAN', 'UP',
       'TIGGO 7', 'JOY', 'TAOS', 'KA', 'SAVEIR

In [33]:
modelos_populares = ['STRADA', 'PICANTO', 'KOMBI', 'FIESTA', 'MOBI', 'CORSA', 'LIVINA', 'CELTA', '207',
                     'VOYAGE', 'KA', 'SAVEIRO', 'PASSAT', 'CLIO', 'SANDERO', 'FOX', 'UNO', 'PRISMA',
                     'POLO', 'PALIO', 'GOL', 'HB20', 'MONTANA', 'KWID', 'ARGO', 'SPACEFOX', 'XC90',
                     'FUSCA', 'HB20','PALIO']
carros_filtrados = dataset_original[dataset_original['modelo'].isin(modelos_populares)]

In [34]:
# Dessa seleção, avaliamos o número de ofertas por UF
carros_filtrados.UF.value_counts()

SP    633
RJ     97
SC     93
PR     85
MG     61
BA     46
RS     37
GO     21
PE     12
PB     10
PA     10
CE      8
AM      6
RN      5
AL      4
MS      4
AC      3
MT      2
TO      2
PI      1
ES      1
Name: UF, dtype: int64

In [35]:
# Passamos então a verificar a média de preço por Estado para os de maior número de ofertas
carros_filtrados.query('UF == "SP"').describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,623.00,633.00,633.00,633.00,633.00,633.00,633.00
mean,10.51,2017.21,2018.13,61003.15,3.84,0.00,81590.76
std,3.51,3.70,2.79,31304.25,0.54,0.00,35036.73
min,8.00,1994.00,2010.00,100.00,2.00,0.00,17651.24
25%,8.00,2015.00,2016.00,33690.00,4.00,0.00,62962.81
50%,8.00,2018.00,2019.00,61173.00,4.00,0.00,73577.41
75%,14.00,2020.00,2020.00,83400.00,4.00,0.00,93629.72
max,20.00,2022.00,2023.00,234329.00,4.00,0.00,364351.72


In [36]:
carros_filtrados.query('UF == "RJ"').describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,97.00,97.00,97.00,97.00,97.00,97.00,97.00
mean,10.62,2016.91,2018.03,61077.14,3.86,0.00,73236.77
std,3.51,4.28,2.79,32658.54,0.52,0.00,17220.69
min,8.00,1997.00,2011.00,6139.00,2.00,0.00,25588.61
25%,8.00,2015.00,2017.00,32982.00,4.00,0.00,61521.53
50%,8.00,2018.00,2019.00,60468.00,4.00,0.00,72981.75
75%,14.00,2020.00,2020.00,83358.00,4.00,0.00,84947.58
max,18.00,2022.00,2022.00,159598.00,4.00,0.00,115563.13


In [37]:
carros_filtrados.query('UF == "SC"').describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,93.00,93.00,93.00,93.00,93.00,93.00,93.00
mean,11.45,2017.14,2018.02,57055.62,3.84,0.00,77489.23
std,3.77,3.63,2.81,30678.01,0.54,0.00,35680.47
min,8.00,2001.00,2011.00,2722.00,2.00,0.00,14613.20
25%,8.00,2017.00,2017.00,32971.00,4.00,0.00,61118.43
50%,8.00,2018.00,2019.00,58342.00,4.00,0.00,70535.73
75%,15.00,2020.00,2020.00,77047.00,4.00,0.00,84349.96
max,20.00,2021.00,2022.00,130721.00,4.00,0.00,307462.88


In [38]:
carros_filtrados.query('UF == "PR"').describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,82.00,85.00,85.00,85.00,85.00,85.00,85.00
mean,10.38,2017.94,2018.85,49543.09,3.91,0.00,88656.41
std,3.45,3.79,2.77,30604.84,0.43,0.00,25731.64
min,8.00,2000.00,2011.00,8720.00,2.00,0.00,29907.53
25%,8.00,2016.00,2017.00,24621.00,4.00,0.00,72312.09
50%,8.00,2019.00,2020.00,46093.00,4.00,0.00,81134.35
75%,14.00,2020.00,2021.00,69178.00,4.00,0.00,106089.60
max,20.00,2021.00,2022.00,139084.00,4.00,0.00,168371.00


In [39]:
carros_filtrados.query('UF == "MG"').describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,61.00,61.00,61.00,61.00,61.00,61.00,61.00
mean,10.54,2017.34,2018.38,59597.03,3.87,0.00,85545.40
std,3.59,3.64,2.48,28089.28,0.50,0.00,29594.03
min,8.00,1998.00,2011.00,7509.00,2.00,0.00,39933.89
25%,8.00,2016.00,2017.00,36254.00,4.00,0.00,66644.00
50%,8.00,2018.00,2019.00,56660.00,4.00,0.00,78381.80
75%,14.00,2020.00,2020.00,75652.00,4.00,0.00,101771.22
max,20.00,2021.00,2022.00,124778.00,4.00,0.00,240173.94


In [40]:
# Após a análise, ficou claro que o Estado com os melhores valores de venda, foi o Paraná, conforme abaixo.
carros_filtrados.query('UF == "PR"').sort_values(by='preco', ascending=False)

,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,tipo_vendedor,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,UF,preco
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16426693425948974575904239705483877397,8.00,VOLKSWAGEN,GOL,1.6 16V MSI TOTALFLEX 4P AUTOMÁTICO,2021,2021.00,20978.00,Automática,4,Sedã,N,Branco,PF,Medianeira,Paraná (PR),Pessoa Física,False,False,False,Aceita troca,Único dono,0,IPVA pago,Licenciado,0,0,0.00,PR,168371.00
212006554751783483790679711119782928872,8.00,VOLKSWAGEN,POLO,1.4 250 TSI GTS AUTOMÁTICO,2020,2021.00,20340.00,Automática,4,Sedã,N,Branco,PF,Belo Horizonte,Paraná (PR),Pessoa Física,False,False,False,NaN,Único dono,0,IPVA pago,Licenciado,Garantia de fábrica,0,0.00,PR,140788.64
241456460944252890637034191955123928779,8.00,VOLKSWAGEN,FUSCA,2.0 TSI 16V GASOLINA 2P AUTOMÁTICO,2015,2016.00,54421.00,Automática,4,Picape,N,Branco,PF,Medianeira,Paraná (PR),Pessoa Física,False,False,False,Aceita troca,,Todas as revisões feitas pela concessionária,IPVA pago,NaN,0,Todas as revisões feitas pela agenda do carro,0.00,PR,139960.81
313999043008824885491867759432949741024,8.00,VOLKSWAGEN,GOL,1.6 16V MSI TOTALFLEX 4P AUTOMÁTICO,2020,2021.00,19685.00,Automática,4,Hatchback,N,Branco,PF,Curitiba,Paraná (PR),Pessoa Física,False,False,False,NaN,Único dono,0,IPVA pago,NaN,0,0,0.00,PR,134106.78
48063633162408713342025457997117248607,9.00,VOLKSWAGEN,GOL,1.6 16V MSI TOTALFLEX 4P AUTOMÁTICO,2020,2021.00,61608.00,Automática,4,Hatchback,N,Prata,PJ,Curitiba,Paraná (PR),Loja,False,False,False,Aceita troca,Único dono,0,IPVA pago,Licenciado,0,0,0.00,PR,134106.78
189969101586220581917589841041074724515,14.00,VOLKSWAGEN,FUSCA,2.0 TSI 16V GASOLINA 2P AUTOMÁTICO,2014,2015.00,83530.00,Automática,4,Picape,N,Branco,PJ,Curitiba,Paraná (PR),Loja,False,False,False,Aceita troca,,0,NaN,NaN,0,0,0.00,PR,130163.70
97106784096228944856446886886525991585,8.00,VOLKSWAGEN,GOL,1.6 16V MSI TOTALFLEX 4P AUTOMÁTICO,2020,2021.00,24621.00,Automática,4,Hatchback,N,Branco,PF,Medianeira,Paraná (PR),Pessoa Física,False,False,False,NaN,Único dono,Todas as revisões feitas pela concessionária,NaN,NaN,Garantia de fábrica,Todas as revisões feitas pela agenda do carro,0.00,PR,128735.41
290197976331230542823844608086952405908,9.00,VOLKSWAGEN,GOL,1.6 16V MSI TOTALFLEX 4P AUTOMÁTICO,2021,2021.00,54089.00,Automática,4,Hatchback,N,Prata,PJ,Belo Horizonte,Paraná (PR),Loja,False,False,False,Aceita troca,Único dono,0,IPVA pago,Licenciado,0,0,0.00,PR,127750.28
114203315066385647574592125513205158030,9.00,VOLKSWAGEN,GOL,1.6 16V MSI TOTALFLEX 4P AUTOMÁTICO,2021,2021.00,54935.00,Automática,4,Hatchback,N,Prata,PJ,Belo Horizonte,Paraná (PR),Loja,False,False,False,Aceita troca,Único dono,0,IPVA pago,Licenciado,0,0,0.00,PR,127750.28


### Qual o melhor Estado para se comprar uma picape com transmissão automática? Por que?
A seleção apontou 1084 picapes com câmbio automático. Utilizando o critério de avaliação de média de preço de venda e ano-modelo mais recente, verificamos que o Estado com o melhor conjunto de ofertas é Santa Catarina (SC)

In [41]:
# Selecionamos as picapes com câmbio automático
picape_boa = dataset_original.query('tipo == "Picape" & cambio == "Automática"').sort_values(by='preco', ascending=False)
picape_boa.shape

(1084, 29)

In [42]:
# Verificamos as UF com ofertas de picapes com câmbio automático
picape_boa.UF.value_counts()

SP    552
SC    121
PR    101
RJ     88
RS     63
MG     61
GO     41
BA     29
PE      7
TO      6
AL      3
MT      3
SE      2
PB      2
AM      2
AC      1
MS      1
RN      1
Name: UF, dtype: int64

In [43]:
# Selecionamos as 6 maiores para avaliação
picape_boa.query('UF == "SP"').describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,552.00,552.00,552.00,552.00,552.00,552.00,552.00
mean,10.50,2016.94,2017.74,55249.62,3.71,0.00,194745.83
std,3.58,3.02,2.43,34268.12,0.70,0.00,80751.92
min,8.00,1995.00,2009.00,100.00,2.00,0.00,67576.96
25%,8.00,2015.00,2016.00,25386.75,4.00,0.00,131100.70
50%,8.00,2017.00,2018.00,49880.50,4.00,0.00,182961.24
75%,15.00,2019.00,2020.00,83339.00,4.00,0.00,234470.46
max,20.00,2021.00,2022.00,192778.00,4.00,0.00,553720.06


In [44]:
picape_boa.query('UF == "SC"').describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,121.00,121.00,121.00,121.00,121.00,121.00,121.00
mean,11.09,2016.83,2017.87,57744.74,3.72,0.00,186300.92
std,3.69,3.99,2.41,37924.60,0.70,0.00,67717.62
min,8.00,1993.00,2011.00,100.00,2.00,0.00,61755.16
25%,8.00,2015.00,2017.00,26207.00,4.00,0.00,141395.06
50%,8.00,2017.00,2018.00,60089.00,4.00,0.00,180149.19
75%,15.00,2019.00,2020.00,83171.00,4.00,0.00,225026.23
max,20.00,2021.00,2021.00,236985.00,4.00,0.00,437870.84


In [45]:
picape_boa.query('UF == "PR"').describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,101.00,101.00,101.00,101.00,101.00,101.00,101.00
mean,10.47,2016.94,2017.73,56115.97,3.62,0.00,196562.73
std,3.90,2.89,2.31,32261.37,0.79,0.00,69116.05
min,8.00,2001.00,2013.00,147.00,2.00,0.00,90796.28
25%,8.00,2015.00,2016.00,27206.00,4.00,0.00,145386.59
50%,8.00,2017.00,2018.00,54421.00,4.00,0.00,181942.86
75%,14.00,2019.00,2019.00,82652.00,4.00,0.00,224487.81
max,20.00,2022.00,2022.00,180545.00,4.00,0.00,442819.69


In [46]:
picape_boa.query('UF == "RJ"').describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,88.00,88.00,88.00,88.00,88.00,88.00,88.00
mean,10.60,2016.61,2017.65,59879.09,3.73,0.00,188438.55
std,3.73,4.08,2.60,36407.66,0.69,0.00,77506.52
min,8.00,1993.00,2011.00,811.00,2.00,0.00,51885.62
25%,8.00,2015.75,2016.75,31093.75,4.00,0.00,139651.48
50%,8.00,2017.00,2018.00,62961.00,4.00,0.00,177901.58
75%,14.25,2019.00,2020.00,84356.00,4.00,0.00,211737.16
max,20.00,2021.00,2022.00,185585.00,4.00,0.00,453174.19


In [47]:
picape_boa.query('UF == "RS"').describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,63.00,63.00,63.00,63.00,63.00,63.00,63.00
mean,9.67,2016.24,2017.25,59285.78,3.68,0.00,205540.25
std,3.07,4.41,2.55,37969.78,0.74,0.00,94223.38
min,8.00,1995.00,2011.00,100.00,2.00,0.00,52865.39
25%,8.00,2016.00,2016.50,26784.50,4.00,0.00,144366.84
50%,8.00,2017.00,2018.00,63329.00,4.00,0.00,190629.97
75%,8.00,2019.00,2019.00,85713.00,4.00,0.00,240048.33
max,17.00,2021.00,2021.00,216133.00,4.00,0.00,463441.66


In [48]:
picape_boa.query('UF == "MG"').describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,veiculo_alienado,preco
count,61.00,61.00,61.00,61.00,61.00,61.00,61.00
mean,9.93,2016.82,2017.75,53888.62,3.70,0.00,213724.05
std,3.60,3.47,2.39,34824.02,0.72,0.00,80878.59
min,8.00,1999.00,2012.00,2333.00,2.00,0.00,65978.03
25%,8.00,2015.00,2016.00,24037.00,4.00,0.00,157152.03
50%,8.00,2017.00,2018.00,41929.00,4.00,0.00,202704.08
75%,8.00,2019.00,2020.00,80670.00,4.00,0.00,248588.42
max,20.00,2021.00,2022.00,142295.00,4.00,0.00,425453.53


### Qual o melhor Estado para se comprar carros com garantia de fábrica? Por que?

#### Qual tipo de problema estamos resolvendo (regressão, classificação)? Explique como você faria a previsão do preço a partir dos dados. Quais variáveis e/ou suas transformações você utilizou e por quê? Qual modelo melhor se aproxima dos dados e quais seus prós e contras? Qual medida de performance do modelo foi escolhida e por quê?
   Primeiro, identificamos quais variáveis no dataset seriam relevantes para a previsão do preço. Verificamos que era necessário realizar algumas transformações nesses dados e iniciamos a Análise Exploratória de dados. Finalizando a EDA, demos início à separação das bases para treinar nos modelos usando a função train_test_split.
   Durante todo o processo de análise exploratória de dados foi possível analisar que o problema apresentado era um problema de regressão, no qual buscamos prever o preço de venda dos carros com base em um conjunto de variáveis contínuas. Para realizar essa previsão, testamos três modelos de regressão: regressão linear, árvore de decisão e XGBoost, dentre os quais escolhemos o modelo XGBoost que, no contexto do problema descrito, parece ser o mais adequado para se aproximar dos dados apresentando os melhores resultados quando aplicado ao conjunto de dados de treinamento.

## Escrevendo em arquivo

In [49]:
# Criação de um dataset com as duas colunas selecionadas
dataset_original.reset_index(inplace=True)
coluna1 = dataset_original['id']
coluna2 = dataset_original['preco']
dataset_gravacao = pd.DataFrame({'id': coluna1, 'preco': coluna2}).set_index('id')

# Salve o novo dataset
dataset_gravacao.to_csv('Predicted.csv')


## Fim